In [1]:
import os
os.chdir('../')
%pwd

'c:\\Users\\jacob\\Desktop\\Technica_Assesment\\Question_2\\Mushroom_Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    train_data_target: Path
    test_data_path :Path
    test_data_target: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [3]:
from Mushroom_Classification_src.constants import *
from Mushroom_Classification_src.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path =config.train_data_path,
            train_data_target= config.train_data_target,
            test_data_path=config.test_data_path,  
            test_data_target = config.test_data_target,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os
from Mushroom_Classification_src import logger
from sklearn.linear_model import ElasticNet
import joblib

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path).values
        train_data_target=pd.read_csv(self.config.train_data_target).values
        




        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_data, train_data_target)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))



In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-11-27 23:46:00,572: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-27 23:46:00,578: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-27 23:46:00,604: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-27 23:46:00,608: INFO: common: created directory at: artifacts]
[2024-11-27 23:46:00,611: INFO: common: created directory at: artifacts/model_trainer]
